In [46]:
from __future__ import division, print_function
import numpy as np
import pandas as pd

In [47]:
from datetime import timedelta

In [48]:
def get_NWRMSLE(predicted_l, actual_l, weights_l):
    p_ = np.array(predicted_l)
    a_ = np.array(actual_l)
    ret_val = np.sqrt(np.sum((weights_l*np.log(p_+1)-np.log(a_+1))**2)/sum(weights_l))
    return ret_val

def to_log(arr):
    arr_ = np.array(arr)
    return np.log(arr_+1)

def from_log(arr):
    arr_ = np.array(arr)
    return np.exp(arr_)-1

In [49]:
root_path = '/home/as/Desktop/machine learning/___kaggle/CorporacionFavorita'

In [50]:
test_df = pd.read_csv(root_path+'/data/test.csv', parse_dates=['date'])#,index_col='id')

### Process all stores

In [51]:
#%%time

test_df.loc[:,'unit_sales'] = 0
scores_ = []

for store_id in range(1,55):
    print('processing store: ',store_id)
        #read store data
    dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32'}
    td_s_exp = pd.read_csv(root_path+'/data/train_slices3/train_data_store'+str(store_id)+'.csv', 
                           dtype=dtypes,parse_dates=['date'])
    #td_s_exp.drop(['Unnamed: 0'],axis=1,inplace=True)
    ws = td_s_exp.loc[(td_s_exp['date']>='2016-08-01')]
    del td_s_exp
        #calculate mean sales
    ws.loc[ws['unit_sales']<0,['unit_sales']] = 0
    ws['dayofweek'] = ws['date'].dt.dayofweek
    ws['unit_sales'] = ws['unit_sales'].apply(pd.np.log1p) #converting to x to log(1+x)
    ma_dw = ws[['item_nbr','dayofweek','unit_sales']].groupby(['item_nbr','dayofweek'])['unit_sales'].mean()\
                                                       .to_frame('madw').reset_index()
    ma_wk = ma_dw[['item_nbr','madw']].groupby(['item_nbr'])['madw'].mean().to_frame('mawk').reset_index()
    
    u_dates = ws.date.unique()
    u_items = ws.item_nbr.unique()
    ws.set_index(['date', 'item_nbr'], inplace=True)
    ws = ws.reindex(pd.MultiIndex.from_product((u_dates, u_items),names=['date','item_nbr'])).reset_index()
    del u_dates, u_items
    ws.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
    
    lastdate = ws.iloc[ws.shape[0]-1].date
    ma_is = ws[['item_nbr','unit_sales']].groupby(['item_nbr'])['unit_sales'].mean().to_frame('mais')
    for i in [112,56,28,14,7,3,1]:
        tmp = ws[ws.date>lastdate-timedelta(int(i))]
        tmpg = tmp.groupby(['item_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
        ma_is = ma_is.join(tmpg, how='left')
    del tmp,tmpg,ws

    ma_is['mais']=ma_is.median(axis=1)
    ma_is.reset_index(inplace=True)
    ma_is.drop(list(ma_is.columns.values)[2:],1,inplace=True) #was [3:]

            #get test slice
    #ws_t = pd.read_csv(root_path+'/data/test_slices/test_slice_'+str(store_id)+'.csv',index_col='Unnamed: 0')
    #ws_t.drop(['outs'+str(sn_) for sn_ in range(1,55,1)],axis=1,inplace=True)
    #ws_t.drop(['int'+str(sn_) for sn_ in range(1,55,1) if sn_!=store_id+100],axis=1,inplace=True)
    ws_t = test_df[test_df['store_nbr']==store_id]
    ws_t['dayofweek'] = ws_t['date'].dt.dayofweek
    ws_t = pd.merge(ws_t, ma_is, how='left', on=['item_nbr'])
    ws_t = pd.merge(ws_t, ma_wk, how='left', on=['item_nbr'])
    ws_t = pd.merge(ws_t, ma_dw, how='left', on=['item_nbr','dayofweek'])
    del ma_is, ma_wk, ma_dw
            #build forecast
    ws_t['unit_sales'] = ws_t.mais 
    pos_idx = ws_t['mawk'] > 0
    test_pos = ws_t.loc[pos_idx]
    ws_t.loc[pos_idx, ['unit_sales']] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
    ws_t.loc[:,"unit_sales"].fillna(0, inplace=True)
    ws_t['unit_sales'] = ws_t['unit_sales'].apply(pd.np.expm1) # restoring 
    ws_t.loc[ws_t['onpromotion']==True, ['unit_sales']] *= 1.5
    ws_t.loc[:,"unit_sales"].fillna(0, inplace=True)
    
    test_df.loc[test_df['store_nbr']==store_id,['unit_sales']] = np.array(ws_t['unit_sales'])
    del ws_t

print('done')
#test_df.to_csv(root_path+'/data/_test_df_processed_20171212.csv', float_format='%.3f',compression='gzip')

processing store:  1


/home/as/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


processing store:  2
processing store:  3
processing store:  4
processing store:  5
processing store:  6
processing store:  7
processing store:  8
processing store:  9
processing store:  10
processing store:  11
processing store:  12
processing store:  13
processing store:  14
processing store:  15
processing store:  16
processing store:  17
processing store:  18
processing store:  19
processing store:  20
processing store:  21
processing store:  22
processing store:  23
processing store:  24
processing store:  25
processing store:  26
processing store:  27
processing store:  28
processing store:  29
processing store:  30
processing store:  31
processing store:  32
processing store:  33
processing store:  34
processing store:  35
processing store:  36
processing store:  37
processing store:  38
processing store:  39
processing store:  40
processing store:  41
processing store:  42
processing store:  43
processing store:  44
processing store:  45
processing store:  46
processing store: 

**Submission to kaggle**

In [52]:
test_df[['id','unit_sales']].to_csv(root_path+'/data/_submission.csv.gz',header=True,index=False,float_format='%.3f',
                                   compression='gzip')

In [53]:
test_df['unit_sales'].describe()

count    3.370464e+06
mean     3.272853e+00
std      1.104077e+01
min      0.000000e+00
25%      0.000000e+00
50%      8.078568e-01
75%      2.895783e+00
max      2.554157e+03
Name: unit_sales, dtype: float64

In [54]:
ppcsv = pd.read_csv(root_path+'/data/ma8dwof.csv')

In [55]:
test_df['unit_sales2'] = ppcsv['unit_sales']

In [60]:
test_df['unit_sales'] = np.round(test_df['unit_sales'],3)